In [2]:
from Library import OpenCVWrapper as CV
sample = CV.Image("coffeebeans.jpg",1)
sample = (
    sample
    .cvtColor("GRAY")
    .gaussianBlur()
    .threshOTSU()
    .bitwiseNot()
    .fillHole()
    .findContours()
    .drawContours(color=(0,255,0))
    .show()
)

In [1]:
%ls

FLAGS4-6-0.json  Library/    SKETCH.ipynb  coffeebeans.jpg  main.py
LICENSE          NOTE.ipynb  __init__.py   core/
